<a href="https://colab.research.google.com/github/delhian/NLP_course/blob/master/Home%20Tasks/final_task_unsolved_mod_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Постановка задачи

В рамках данного финального задания мы посторим сеть для перевода с немецкого языка на английский.

*Этот блокнот основан на [реализации с открытым исходным кодом](https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb) модели seq2seq NMT в PyTorch.*

Мы попробуем реализовать модель из этой статьи -> [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215).

Модель будет обучена переводу с немецкого на английский, но ее можно применить к любой проблеме, связанной с переходом от одной последовательности к другой, например, к обобщению.

Наиболее распространенными моделями последовательностей (seq2seq) являются модели *энкодер-декодер*, которые (обычно) используют *рекуррентную нейронную сеть* (RNN) для *кодирования* исходного (входного) предложения в один вектор. В этом ноутбуке мы будем называть этот единственный вектор *контекстным вектором*. Вы можете рассматривать вектор контекста как абстрактное представление всего входного предложения. Этот вектор затем *декодируется* вторым RNN, который учится выводить целевое (выходное) предложение, генерируя его по одному слову за раз.

![](https://github.com/neychev/made_nlp_course/blob/master/week06_SelfAttention_and_NMT_practice/img/seq2seq1.png?raw=1)

In [ ]:
!pip install https://download.pytorch.org/whl/nightly/cu102/torch-1.8.0.dev20210210-cp37-cp37m-linux_x86_64.whl
!pip install torchtext==0.9.0

     |███████████████████████████████▏| 834.1 MB 1.2 MB/s eta 0:00:19tcmalloc: large alloc 1147494400 bytes == 0x55e7b5cb6000 @  0x7f234d753615 0x55e77bf344cc 0x55e77c01447a 0x55e77bf372ed 0x55e77c028e1d 0x55e77bfaae99 0x55e77bfa59ee 0x55e77bf38bda 0x55e77bfaad00 0x55e77bfa59ee 0x55e77bf38bda 0x55e77bfa7737 0x55e77c029c66 0x55e77bfa6daf 0x55e77c029c66 0x55e77bfa6daf 0x55e77c029c66 0x55e77bfa6daf 0x55e77bf39039 0x55e77bf7c409 0x55e77bf37c52 0x55e77bfaac25 0x55e77bfa59ee 0x55e77bf38bda 0x55e77bfa7737 0x55e77bfa59ee 0x55e77bf38bda 0x55e77bfa6915 0x55e77bf38afa 0x55e77bfa6c0d 0x55e77bfa59ee
     |████████████████████████████████| 857.4 MB 5.2 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchv

In [ ]:
!python -m spacy download en > /dev/null
!python -m spacy download de > /dev/null

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
from torchtext.data.utils import get_tokenizer
import io

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

import spacy

from collections import Counter
import random
import math
import time

Зафиксируем все возможные случайные сиды, для максимальной воспроизводимости

In [ ]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.de.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

In [ ]:
train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

train.de.gz: 100%|██████████| 637k/637k [00:00<00:00, 8.33MB/s]
train.en.gz: 100%|██████████| 569k/569k [00:00<00:00, 11.3MB/s]
val.de.gz: 100%|██████████| 24.7k/24.7k [00:00<00:00, 7.91MB/s]
val.en.gz: 100%|██████████| 21.6k/21.6k [00:00<00:00, 4.50MB/s]
test_2016_flickr.de.gz: 100%|██████████| 22.9k/22.9k [00:00<00:00, 5.54MB/s]
test_2016_flickr.en.gz: 100%|██████████| 21.1k/21.1k [00:00<00:00, 9.08MB/s]


In [ ]:
train_filepaths

['/content/.data/train.de', '/content/.data/train.en']

Проинициализируем токенизаторы для исходного и целевого языка:

In [ ]:
de_tokenizer = get_tokenizer('spacy', language='de')
en_tokenizer = get_tokenizer('spacy', language='en')

## Загрузка данных

Напишите функцию по созданию словаря, возвращающую словарь из `torchtext`:

In [ ]:
def build_vocab(filepath, tokenizer):
  counter = Counter()
  with io.open(filepath, encoding="utf8") as f:
    for string_ in f:
      counter.update(tokenizer(string_))
  return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])


In [ ]:
de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

Проверка функции:

In [ ]:
assert en_vocab['a'] < 10
assert de_vocab['Ein'] < 15000

Напишем функцию для составления большого тензора, который будет хранить на каждой строчке исходное токенизированное предложение и целевое.

In [ ]:
def data_process(filepaths):
  raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
  raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
  data = []
  for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
    de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de)],
                            dtype=torch.long)
    en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)],
                            dtype=torch.long)
    data.append((de_tensor_, en_tensor_))
  return data

Проинициализируйте переменные `train_data, val_data, test_data`.

In [ ]:
# YOUR CODE HERE

train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

In [ ]:
assert len(train_data) == 29000
assert len(val_data) == 1014
assert len(test_data) == 1000
assert train_data[23][0].sum().item() > 3220

In [ ]:
#YOUR CODE HERE

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
assert device is not None
assert device.type == 'cuda'

In [ ]:
BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

Дополните функцию `generate_batch` чтобы она добавляла паддинг в конце и чтобы первыми символами были `BOS_IDX`, а последним `EOS_IDX`.

In [ ]:
# YOUR CODE HERE

def generate_batch(data_batch):
  de_batch, en_batch = [], []
  for (de_item, en_item) in data_batch:
    de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return de_batch, en_batch

In [ ]:
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

In [ ]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

## Подготовка модели

Наконец мы переходим к написанию самой нейросети. Она будет состоять из 3 различных классов, а финальная сеть будет состоять из этих трех классов. Получится несколько объемно, но когда мы начинаем переходить к более серьезным сетям.

Создадим модель Seq2Seq на основе GRU:

Создайте класс `Encoder`, который состоит из слоя Embedding, GRU(bidirectional) и Linear:

In [ ]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden

Оставим класс Attention, чтобы сохранить преемственность с оригинальной статьей. Подбробнее про Attention мы поговорим в следующих модулях.

In [ ]:
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)

Добавим класс `Decoder`.

Аналогично классу `Encoder` сделайте класс `Decoder` таким же по архитектуре, однако линейный слой будет иметь размерность emb_dim  плюс размерность выходного слоя Attention.

In [ ]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)

Наконец, объединим все в класс модели `Seq2Seq`.

Напишите, что должен принимать на вход класс Seq2Seq и проинициализируйте экземпляр этого класса:

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


Вы можете попробовать тут свои значения для разных слоев. Единственное - правильно задайте размеры входных слоев для Encoder и Decoder

In [ ]:
INPUT_DIM = len(de_vocab)
OUTPUT_DIM = len(en_vocab)
ENC_EMB_DIM = 64
DEC_EMB_DIM = 64
ENC_HID_DIM = 128
DEC_HID_DIM = 128
ATTN_DIM = 32
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

In [ ]:
model = Seq2Seq(enc, dec, device).to(device)

Напишите функцию, считающую параметры модели:

In [ ]:
def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
assert count_parameters(model) > 7_000_000

Проинициализируем веса, как это рекомендуют в оригинальной статье:

In [ ]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(19206, 64)
    (rnn): GRU(64, 128, bidirectional=True)
    (fc): Linear(in_features=256, out_features=128, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=384, out_features=32, bias=True)
    )
    (embedding): Embedding(10840, 64)
    (rnn): GRU(320, 128)
    (out): Linear(in_features=448, out_features=10840, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
PAD_IDX = en_vocab.stoi['<pad>']

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

## Обучение модели

Напишите функцию обучения модели. Если получится, добавьте в тренировочный цикл запись значений функции потерь и перплексии в tensorboard.

In [ ]:
import math
import time


def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

Напишем функцию для валидации модели. Хочу отметить, что при валидации наиболее важным моментом является выключение метода teacher-forcing. 

In [ ]:


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [ ]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:


N_EPOCHS = 50
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Epoch: 01 | Time: 1m 25s
	Train Loss: 2.210 | Train PPL:   9.112
	 Val. Loss: 3.697 |  Val. PPL:  40.317
Epoch: 02 | Time: 1m 25s
	Train Loss: 2.141 | Train PPL:   8.511
	 Val. Loss: 3.700 |  Val. PPL:  40.464
Epoch: 03 | Time: 1m 25s
	Train Loss: 2.085 | Train PPL:   8.044
	 Val. Loss: 3.719 |  Val. PPL:  41.238
Epoch: 04 | Time: 1m 24s
	Train Loss: 2.016 | Train PPL:   7.509
	 Val. Loss: 3.753 |  Val. PPL:  42.629
Epoch: 05 | Time: 1m 24s
	Train Loss: 1.981 | Train PPL:   7.249
	 Val. Loss: 3.696 |  Val. PPL:  40.301
Epoch: 06 | Time: 1m 23s
	Train Loss: 1.945 | Train PPL:   6.997
	 Val. Loss: 3.700 |  Val. PPL:  40.456
Epoch: 07 | Time: 1m 24s
	Train Loss: 1.890 | Train PPL:   6.621
	 Val. Loss: 3.722 |  Val. PPL:  41.358
Epoch: 08 | Time: 1m 24s
	Train Loss: 1.845 | Train PPL:   6.326
	 Val. Loss: 3.734 |  Val. PPL:  41.840
Epoch: 09 | Time: 1m 23s
	Train Loss: 1.792 | Train PPL:   6.004
	 Val. Loss: 3.797 |  Val. PPL:  44.557
Epoch: 10 | Time: 1m 24s
	Train Loss: 1.793 | Train PPL

Добейтесь перплексии на валидационной выборке порядка 35-40. Попробуйте различные значения параметров сети и learning rate.